# Exploring command line features

Various command line features are able to change the EHub objective function, solver options and output presentation.

In [1]:
import argparse
import pickle
import yaml
from pyehub.energy_hub.ehub_model import EHubModel
from pyehub.logger import create_logger
from pyehub.outputter import pretty_print, output_excel, print_capacities, print_section, print_warning, stream_info
from pyehub.energy_hub.utils import constraint

Here you set the values of all the variables that would be passed as command line arguments in the terminal.

In [2]:
DEFAULT_LOG_FILE = 'logs.log'

settings = {
    # Suppresses all output
    'quiet': False,
    # Output everything
    'verbose': False,
    # The Excel file of the model to solve
    'input_file': 'test_file.xlsx',
    # The file to output the results in
    'output_file': 'test_out.xlsx',
    # Add a maximum value to the carbon
    'carbon': True,
    # The maximum value that carbon can have if the --carbon switch is used.
    'max_carbon': 5,
    'solver_options': False,
    # The solver to use', choices=['glpk', 'cplex']
    'solver': {
                'name': 'glpk',
                'options': {
                    'mipgap': 0.05,
                },
              },
    # Outputs the results in a different format, grouped info about streams
    'output_format': False
}

Adding a carbon constraint is a common problem for the EHub. `CarbonEHubModel(EHubModel)` is a version of the EHub with a `max_carbon_level(self)` constraint limiting carbon emissions of the model. Below is added constraint:

In [3]:
class CarbonEHubModel(EHubModel):
    MAX_CARBON = 0

    def __init__(self, *, excel=None, request=None, max_carbon=0):
        super().__init__(excel=excel, request=request)
        if max_carbon is not None:
            self.MAX_CARBON = max_carbon

    @constraint()
    def max_carbon_level(self):
        return self.total_carbon <= float(self.MAX_CARBON)


The main() function is where the EHub is defined.

In [4]:
def main():
    create_logger(DEFAULT_LOG_FILE)
    if settings['carbon']:
        model = CarbonEHubModel(excel=settings['input_file'], max_carbon=settings['max_carbon'])
    else:
        model = EHubModel(excel=settings['input_file'])

    # Printing out model prior to solving in the same format as after solving
    if settings['verbose']:
        print_section('Before_Solving', model._public_attributes())

    results = model.solve(settings['solver'], is_verbose=settings['verbose'])

    if not settings['quiet']:
        pretty_print(results)

    output_excel(results['solution'], settings['output_file'],
                 time_steps=len(model.time), sheets=['other', 'capacity_storage', 'capacity_tech'])

    # Outputs all the capacities form the results
    if settings['verbose']:
        print_capacities(results)

    # Prints error at the end if the model burns energy(energy from storage and energy to storage at the same time step)
    print_warning(results)

    # Outputs additional sheets for each stream with information about storages and about the stream
    if settings['output_format']:
        stream_info(results, settings['output_file'])

In [5]:
#Execute main()
if __name__ == "__main__":
    main()

Version: 0.1.0
Solver
	termination condition: Optimal
	time: 0.024825357999361586
Solution

========== Stuff ==========

ANNUAL_MAINTENANCE_STORAGE_COSTS: 
                ANNUAL_MAINTENANCE_STORAGE_COSTS
Battery                                        0
Hot Water Tank                                 0

BIG_M: 
99999

Battery: 
0.0

Boiler: 
10.0

CARBON_CREDITS: 
            CARBON_CREDITS
Elec                      
Gas                       
Grid                      
Heat                      
Irradiation               
PV_Elec                0.3

CARBON_FACTORS: 
            CARBON_FACTORS
Elec                      
Gas                  0.194
Grid                  0.35
Heat                      
Irradiation               
PV_Elec                   

CHARGING_EFFICIENCY: 
                CHARGING_EFFICIENCY
Battery                        0.99
Hot Water Tank                 0.80

CHP: 
5.0

CONVERSION_EFFICIENCY: 
          Elec  ...  PV_Elec
Boiler     0.0  ...    0.000
CHP        0.